<a href="https://colab.research.google.com/github/HarshKapr/Cloud/blob/main/Assesment2_SIG742_Harsh_Pavan_Team19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necessary libraries
!pip install pyspark pandas matplotlib seaborn
# Import required libraries


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=aa1ad846d284365c4da54df20d6d2afbee9ddd7d6cdce8466dce913413abf87d
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Initialize Spark session
spark = SparkSession.builder.appName("TransactionAnalysis").getOrCreate()

**Task 1: Data Wrangling and Initial Processing**

**1.1.1 Handling 'NA' values in `CustomerNo`**

In [10]:
# Read the CSV data
data_path = "C:/Users/Harsh Kapoor/Downloads/transactionrecord.csv"
df = spark.read.csv(data_path, header=True, inferSchema=True)
# Replace 'NA' in CustomerNo with '-1'
df = df.withColumn("CustomerNo", when(df["CustomerNo"] == "NA", "-1").otherwise(df["CustomerNo"]))
# Show the updated DataFrame
df.show(5)

Py4JJavaError: An error occurred while calling o66.csv.
: org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "C"
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3443)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:724)
	at scala.collection.immutable.List.map(List.scala:293)
	at org.apache.spark.sql.execution.datasources.DataSource$.checkAndGlobPathIfNecessary(DataSource.scala:722)
	at org.apache.spark.sql.execution.datasources.DataSource.checkAndGlobPathIfNecessary(DataS

In [8]:
# Remove non-alphabet characters from productName
df = df.withColumn("productName_process", regexp_replace(col("productName"), "[^A-Za-z]", ""))
# Show the first 5 rows with the new column
df.select("productName", "productName_process").show(5)

NameError: name 'df' is not defined